# Data preparation final approach

## Overview
1. Read in files received from 23andMe
2. Fasta - generate correct REF allele
3. Annotation
4. Remove synonymous variants and benign/likely benign
5. Fix AA change for 23andMe file
6. Merge annotated clean file with original values
7. Extract info from 23andMe
8. Extract variants from UKB and AMP-PD
9. Association testing
10. Meta-analysis
11. Clean files for analysis

## 1. Read in files from 23andMe

## 2. Fasta - generate correct REF allele

Now run it with the fasta file
(had to download the correct file according to this https://lh3.github.io/2017/11/13/which-human-reference-genome-to-use)

## 3. Annotation

Write file for annotation

In [ ]:
cut -f 1,2,3,7,8 963_snps_alleles_correct_fasta.txt > for_annotation.txt

tail -n +2 for_annotation.txt > for_annotation_nohead.txt

Annotate

In [ ]:
module load annovar

table_annovar.pl for_annotation_nohead.txt $ANNOVAR_DATA/hg38/ -buildver hg38 -protocol refGene,avsnp150,clinvar_20220320,gnomad211_genome -operation g,f,f,f -outfile 963_23andMe_annotated_all -nastring .

Check file in R

## 4. Remove synonymous variants and benign/likely_benign for certain genes

In [ ]:
# Parkinson's
anti %>% filter(grepl("arkinson", CLNDN)) %>% tally()
    n
1 281

anti %>% filter(grepl("arkinson", CLNDN)) %>% group_by(CLNSIG) %>% tally()
# A tibble: 12 × 2
CLNSIG	n
<chr>	<int>
Benign	34
Benign/Likely_benign	18
Conflicting_interpretations_of_pathogenicity	43
Conflicting_interpretations_of_pathogenicity|_risk_factor	1
Likely_benign	29
Likely_pathogenic	4
Pathogenic	39
Pathogenic/Likely_pathogenic	6
Pathogenic/Likely_pathogenic|_risk_factor	1
Pathogenic|_risk_factor	1
risk_factor	6
Uncertain_significance	99

#another 52 to be removed


remove2=anti %>% filter(grepl("arkinson", CLNDN) & CLNSIG == "Benign" | grepl("arkinson", CLNDN) & CLNSIG == "Benign/Likely_benign" | grepl("arkinson", CLNDN) & CLNSIG == "Likely_benign")

anti2 = anti_join(anti, remove2)
dim(anti2)
[1] 839  33

anti2 %>% filter(grepl("arkinson", CLNDN)) %>% group_by(CLNSIG) %>% tally()

#A tibble: 9 × 2
CLNSIG	n
<chr>	<int>
Conflicting_interpretations_of_pathogenicity	43
Conflicting_interpretations_of_pathogenicity|_risk_factor	1
Likely_pathogenic	4
Pathogenic	39
Pathogenic/Likely_pathogenic	6
Pathogenic/Likely_pathogenic|_risk_factor	1
Pathogenic|_risk_factor	1
risk_factor	6
Uncertain_significance	99

In [ ]:
# Write file with chr, start, rsID, REF, ALT

cut -f 1,2,4,5,11 834_23andMe_rare_variants_annotated_fullfile.txt > 834_23andMe_rare_variants_annotated_short.txt

In [ ]:
head 834_23andMe_rare_variants_annotated_short.txt

## 5. Fix AA change for 23andMe file only

In [ ]:
Use this file: 834_23andMe_rare_variants_annotated_fullfile.txt

and this script: AA_change_column.ipynb
# these chunks are only the differences to the above mentioned script

#write.table(fulljoin2, "NM_transcript_list.txt.txt", row.names=F, sep = "\t", quote = F)

merge_empty %>% filter(AAChange != ".")

In [ ]:
# write.table(leftjoin, "Edited_AAChange_23andMe_834_clean.txt", quote = F, sep = "\t", row.names = F) # this has all the VariantNames automatically edited to only one name
wc -l Edited_AAChange_23andMe_834_clean.txt

## 6. Merge annotated clean file with original values

In [ ]:
head variants_to_grep.txt

## 7. Extract info from 23andMe sumstats

## 8. Extract variants from UKB and AMP

In [ ]:
# Write file for that
head variants_to_grep.txt
#cut -f 2 23andMe_834variants_annotated_and_stats.txt > variants_to_grep.txt
# format chr1:bp:REF:ALT

In [ ]:
tail -n +2 variants_to_grep.txt > variants_to_grep_nohead.txt
wc -l variants_to_grep_nohead.txt

head variants_to_grep_nohead.txt

### 8.1 UKB

In [ ]:
# Clean UKB first

#mkdir UKB_and_AMP
# cd ./UKB_and_AMP
# UKB data contains indels
# PLINK file merged with no relateds
# /data/CARD/UKBIOBANK/EXOME_DATA_200K/PVCF_FILES/MERGED_UKB_first_pass.*

In [ ]:
# these files are in plink2 format - convert

## .psam IDs
#IID	SEX
#-000001	NA
#-000002	NA

module load plink/2
plink2 --pfile /data/CARD/UKBIOBANK/EXOME_DATA_200K/PVCF_FILES/MERGED_UKB_first_pass --make-bed --out MERGED_UKB_first_pass

wc -l MERGED_UKB_first_pass.bim
# 16285684 MERGED_UKB_first_pass.bim


# Remove potential indels from UKB
## Write "no_indel" file
#In bash
awk 'length($NF)==1 && length($(NF-1))==1' MERGED_UKB_first_pass.bim > MERGED_UKB_no_indels.txt
wc -l MERGED_UKB_no_indels.txt
# 14908659 MERGED_UKB_no_indels.txt

awk 'length($NF)>1 || length($(NF-1))>1' MERGED_UKB_first_pass.bim > UKB_indels.txt
wc -l UKB_indels.txt
# 1377025 UKB_indels.txt

cut -f 2 MERGED_UKB_no_indels.txt > UKB_noindels_tokeep.txt
module load plink

plink --bfile MERGED_UKB_first_pass --extract UKB_noindels_tokeep.txt --make-bed --out MERGED_UKB_no_indels
# 14908659 variants and 200648 people pass filters and QC.
# Note: No phenotypes present.

# new working files: MERGED_UKB_no_indels

In [ ]:
# Grep variants

In [ ]:
# files written to ./UKB_and_AMP
head variants_to_grep_nohead.txt

grep -w -f variants_to_grep_nohead.txt MERGED_UKB_no_indels.bim > UKB_variants_extracted_from23andMe.txt

wc -l UKB_variants_extracted_from23andMe.txt
# 608 variants

#### 8.1.1 Write binary file

In [ ]:
module load plink/1.9.0-beta4.4

plink --bfile MERGED_UKB_no_indels --extract UKB_variants_extracted_from23andMe.txt --make-bed --out UKB_608_from23andme

# 608 variants and 200648 people pass filters and QC.
# Note: No phenotypes present.


In [ ]:
head UKB_608_from23andme.fam

In [ ]:
# This file needs to be fixed because of an odd format where column 1 is all zeros (column 1 & 2 need to be same)
#cd ./UKB_and_AMP/
cut -d " " -f 2 UKB_608_from23andme.fam > column2.txt
cut -d " " -f 2,3,4,5,6 UKB_608_from23andme.fam > column23456.txt
scp UKB_608_from23andme.fam UKB_608_from23andme_ORIGINAL.fam
paste column2.txt column23456.txt > UKB_608_from23andme.fam

In [ ]:
head UKB_608_from23andme.fam

#### 8.1.2 Write frequency file

In [ ]:
module load plink/1.9.0-beta4.4
plink --bfile UKB_608_from23andme --freq --out UKB_608_from23andme
# Total genotyping rate is 0.998984.
# 200648 people (0 males, 0 females, 200648 ambiguous) loaded from .fam.
#Ambiguous sex IDs written to UKB_644_from23andme.nosex .

In [ ]:
head UKB_608_from23andme.frq

#### 8.1.3 Cohort age per group

In [ ]:
module load plink/1.9.0-beta4.4
plink --bfile UKB_608_from23andme --allow-no-sex --pheno updatePheno_UKB.txt --keep updatePheno_UKB.txt  --make-bed --out UKB_608_from23andme_PhenoUpdate
# 608 variants and 45857 people pass filters and QC.
# Among remaining phenotypes, 7806 are cases and 38051 are controls.

In [ ]:
head UKB_608_from23andme_PhenoUpdate.fam
# this step also removes weird IDs

In [ ]:
head /data/CARD/UKBIOBANK/PHENOTYPE_DATA/disease_groups/UKB_EXOM_ALL_PD_PHENOTYPES_CONTROL_2021_with_PC.txt

### 8.2 AMP-PD

#### 8.2.1 Write binary files

In [ ]:
grep -w -f variants_to_grep_nohead.txt /data/CARD/PD/AMP_NIH/no_relateds/DALGB_12MAR2022/AMPv2.5_samplestoKeep_EuroOnly_noDups_noNIHDups_wPheno_wSex_no_cousins.bim > AMPonly_variants_extracted_from23andMe.txt
wc -l AMPonly_variants_extracted_from23andMe.txt
# 282 AMPonly_variants_extracted_from23andMe.txt

In [ ]:
# Write new binary files
#module load plink/1.9.0-beta4.4

plink --bfile /data/CARD/PD/AMP_NIH/no_relateds/DALGB_12MAR2022/AMPv2.5_samplestoKeep_EuroOnly_noDups_noNIHDups_wPheno_wSex_no_cousins \
--extract AMPonly_variants_extracted_from23andMe.txt \
--make-bed \
--out AMP_282_from23andme

# 282 variants and 4007 people pass filters and QC.
# Among remaining phenotypes, 1451 are cases and 2556 are controls.
# --make-bed to AMP_282_from23andme.bed + AMP_282_from23andme.bim +
# AMP_282_from23andme.fam ... done.
# AMP only contains 282 out of x variants provided by 23andme

#### 8.2.2 Generate frequency file

In [ ]:
# module load plink/1.9.0-beta4.4
plink --bfile AMP_282_from23andme --freq --out AMP_282_from23andme

# 4007 people (2197 males, 1810 females) loaded from .fam.
# 4007 phenotype values loaded from .fam.

#### 8.2.3 Cohort age per group

In [ ]:
# Sex (percentage)

# binary pheno
join_1 %>% group_by(PD_PHENO, SEX) %>% tally() %>% mutate(perc =n/sum(n)*100)

## 9. Association testing

### 9.1 Association testing in Plink

#### 9.1.1 UKB

In [ ]:
module load plink/1.9.0-beta4.4
plink --bfile UKB_608_from23andme --assoc --pheno /data/CARD/UKBIOBANK/PHENOTYPE_DATA/disease_groups/UKB_EXOM_ALL_PD_PHENOTYPES_CONTROL_2021_with_PC.txt --pheno-name PHENO --allow-no-sex --out UKB_608_from23andme_ALL_PD


In [ ]:
# generate raw file for case/control counts

module load plink/1.9.0-beta4.4
plink --bfile UKB_608_from23andme_PhenoUpdate --model --out UKB_608_from23andme_PhenoUpdate --allow-no-sex --pheno /data/CARD/UKBIOBANK/PHENOTYPE_DATA/disease_groups/UKB_EXOM_ALL_PD_PHENOTYPES_CONTROL_2021_with_PC.txt --pheno-name PHENO

In [ ]:
head UKB_608_from23andme_PhenoUpdate.model

In [ ]:
head UKB_608_from23andme_PhenoUpdate_GENO.model
wc -l UKB_608_from23andme_PhenoUpdate_GENO.model

#### 9.1.2 AMP-PD

In [ ]:
plink --bfile AMP_282_from23andme --assoc --pheno /data/CARD/PD/AMP_NIH/no_relateds/COV_PD_NIH_AMPv2.5_samplestoKeep_EuroOnly_noDups_noNIHDups_wPheno_wSex_no_cousins.txt --pheno-name PD_PHENO --out /data/CARD/projects/Rare_variants_2023_VP/AMP_282_from23andme_pheno


In [ ]:
# generate raw file for case/control counts

module load plink/1.9.0-beta4.4
plink --bfile AMP_282_from23andme --model --out AMP_282_from23andme_pheno --pheno /data/CARD/PD/AMP_NIH/no_relateds/COV_PD_NIH_AMPv2.5_samplestoKeep_EuroOnly_noDups_noNIHDups_wPheno_wSex_no_cousins.txt --pheno-name PD_PHENO


In [ ]:
head AMP_282_from23andme_pheno_GENO.model
wc -l AMP_282_from23andme_pheno_GENO.model

### 9.2 Association testing in Rvtest

#### 9.2.1 UKB

In [ ]:
# Convert binary files to vcf for input
module load plink/2.0-dev-20191128
module load samtools
VARIANT_FILE=$1
OUTNAME=${VARIANT_FILE/".txt"/""}
plink2 --bfile UKB_608_from23andme \
--export vcf bgz id-paste=iid --out UKB_608_from23andme${OUTNAME} --mac 1

tabix -p vcf  UKB_608_from23andme${OUTNAME}.vcf.gz

In [ ]:
# Now run rvtest
module load rvtests

rvtest --inVcf UKB_608_from23andme.vcf.gz --pheno /data/CARD/UKBIOBANK/PHENOTYPE_DATA/disease_groups/UKB_EXOM_ALL_PD_PHENOTYPES_CONTROL_2021_with_PC.txt --pheno-name PHENO --out UKB_608_from23andme_withcovars_score_ALL_PD --single wald,score --covar /data/CARD/UKBIOBANK/PHENOTYPE_DATA/disease_groups/UKB_EXOM_ALL_PD_PHENOTYPES_CONTROL_2021_with_PC.txt --covar-name GENETIC_SEX,AGE_OF_RECRUIT,TOWNSEND,PC1,PC2,PC3,PC4,PC5


#### 9.2.2 AMP-PD

In [ ]:
# convert binary plink files to vcf for rvtest
module load plink/2.0-dev-20191128
module load samtools
VARIANT_FILE=$1
OUTNAME=${VARIANT_FILE/".txt"/""}
plink2 --bfile AMP_282_from23andme \
--export vcf bgz id-paste=iid --out AMP_282_from23andme${OUTNAME} --mac 1

tabix -p vcf AMP_282_from23andme${OUTNAME}.vcf.gz


In [ ]:
# new files
AMP_282_from23andme.vcf.gz
AMP_282_from23andme.vcf.gz.tbi

In [ ]:
# Run Rvtest
module load rvtests

# don't correct for age in AMP
rvtest --inVcf AMP_282_from23andme.vcf.gz --pheno /data/CARD/PD/AMP_NIH/no_relateds/COV_PD_NIH_AMPv2.5_samplestoKeep_EuroOnly_noDups_noNIHDups_wPheno_wSex_no_cousins.txt --pheno-name PD_PHENO --out AMP_256_from23andme_withcovars_score --single wald,score --covar /data/CARD/PD/AMP_NIH/no_relateds/COV_PD_NIH_AMPv2.5_samplestoKeep_EuroOnly_noDups_noNIHDups_wPheno_wSex_no_cousins.txt --covar-name SEX,PC1,PC2,PC3,PC4,PC5

## 10. Meta-analysis

### 10.1 Prepare files to match

#### 10.1.1 23andMe

In [ ]:
head 23andMe_834variants_annotated_and_stats.txt

In [ ]:
head toMETA_23andme_summary.txt

#### 10.1.2 UKB

In [ ]:
head toMETA_SCORE_UKBALL.txt

#### 10.1.3 AMP-PD

In [ ]:
head toMETA_SCORE_AMP.txt

### 10.2 Check ID overlap

In [ ]:
cut -f 1 toMETA_23andme_summary.txt > 23andMe_IDs.txt
cut -f 1 toMETA_SCORE_AMP.txt > AMP_IDs.txt
cut -f 1 toMETA_SCORE_UKBALL.txt > UKB_IDs.txt

In [ ]:
# they're all in the same layout, let's merge them and see how much we have left

cat 23andMe_IDs.txt AMP_IDs.txt UKB_IDs.txt > merged_IDs_all3datasets.txt
sort merged_IDs_all3datasets.txt | uniq > merged_IDs_all3datasets_nodupli.txt

wc -l merged_IDs_all3datasets_nodupli.txt
# 780 merged_IDs_all3datasets_nodupli.txt
head merged_IDs_all3datasets_nodupli.txt

# there's an overlap of 780 ID's for all studies, IF all rows have info, i.e. stderr and pvalue etc

In [ ]:
## Check LRRK2 p.G2019S
head -1 toMETA_SCORE_AMP.txt
grep chr12:40340400 toMETA_SCORE_AMP.txt

In [ ]:
head -1 toMETA_SCORE_UKBALL.txt
grep chr12:40340400 toMETA_SCORE_UKBALL.txt

In [ ]:
head -1 toMETA_23andme_summary_VP_new.txt
grep chr12:40340400 toMETA_23andme_summary_VP_new.txt

### 10.3 Create METAL file

In [ ]:
pwd

In [ ]:
cat my_METAL.txt

### 10.4 Run METAL

In [ ]:
module load metal
metal my_METAL.txt

#### 10.4.1 Check warning messages

AMP-PD

UKB

23andMe

In [ ]:
head -1 MY_META_AMP_UKB_23andme1.tbl
grep chr12:40340400 MY_META_AMP_UKB_23andme1.tbl
grep chr1:16986248 MY_META_AMP_UKB_23andme1.tbl
grep chr15:61910283 MY_META_AMP_UKB_23andme1.tbl
grep chr15:61967438 MY_META_AMP_UKB_23andme1.tbl
grep chr3:184319745 MY_META_AMP_UKB_23andme1.tbl

In [ ]:
pwd

## 11. Clean files for analysis

In [2]:
head -n 1 Meta_results_679_AMP_UKB_23andMe_annotation.txt
grep E365K Meta_results_679_AMP_UKB_23andMe_annotation.txt

MarkerName	Allele1	Allele2	Freq1	FreqSE	MinFreq	MaxFreq	Effect	StdErr	P-value	Direction	HetISq	HetChiSq	HetDf	HetPVal	End	Ref	Alt	Func.refGene	Gene	GeneDetail.refGene	ExonicFunc.refGene	AAChange	avsnp150	CLNALLELEID	CLNDN	CLNDISDB	CLNREVSTAT	CLNSIG	AF	AF_popmax	AF_male	AF_female	AF_raw	AF_afr	AF_sas	AF_amr	AF_eas	AF_nfe	AF_fin	AF_asj	AF_oth	non_topmed_AF_popmax	non_neuro_AF_popmax	non_cancer_AF_popmax	controls_AF_popmax	VariantName	CHR.BP
chr1:155236376	t	c	0.9876	0.0012	0.9851	0.9882	-0.3632	0.0342	2.628e-26	---	53.9	4.338	2	0.1143	155236376	C	T	exonic	GBA	.	nonsynonymous SNV	GBA:NM_001171811:exon7:c.G832A:p.E278K,GBA:NM_001171812:exon7:c.G946A:p.E316K,GBA:NM_000157:exon8:c.G1093A:p.E365K,GBA:NM_001005741:exon9:c.G1093A:p.E365K,GBA:NM_001005742:exon9:c.G1093A:p.E365K	rs2230288	.	.	.	.	.	0.0129	0.0141	0.0115	0.0147	0.0129	0.0025	.	0.0035	0	0.0141	0.0400	0.0069	0.0193	0.0138	0.0139	.	0.0134	GBA_E365K	chr1:155236376


Now move files to local and make plots

## 12. Add identifier for variants passing of failing internal QC

In [ ]:
## add identifier to initial file

In [ ]:
scp pitzv2@biowulf.nih.gov:/data/CARD/projects/23andme_annotation/Rare_variant_project_VP/Meta_results_679_AMP_UKB_23andMe_annotation_QC.txt /Users/pitzv2/Documents/Projects/23andme_rare_variants/Final_Dec_2022/Sumstats

## 13. Additional information for revision

### 13.1 Add cases and control minor allele counts (MACs) per variant

### 13.2 Imputation vs genotyped

### 13.3 Write suppl table with frequencies from gnoMad

### 13.4 Estimation of penetrance

In [ ]:
# https://www.cureffi.org/2016/10/19/estimation-of-penetrance/
# penetrance is the probability of disease given a particular genotype. The lower the penetrance of a variant, the higher the frequency.
# if penetrance is quite low (maybe less than 10%), cinical utility of that variant is also low.
# almost all variants with clinical utility have freq <0.001% - more common variants contribute cumulatively little, if any, risk.

In [ ]:
head -n 1 results_combined_23andMe.txt

### 13.5 Haplotype counts for LD on AMP-PD + UKB data (not possible for 23andMe)

In [1]:
# LD estimate in UKB
module load plink/1.9.0-beta4.4
plink --bfile UKB_608_from23andme_PhenoUpdate --allow-no-sex --r2 --ld 'chr1:155238570:C:G' 'chr1:155236376:C:T' --out UKB

[+] Loading plink  1.9.0-beta4.4  on cn4318 
PLINK v1.90b4.4 64-bit (21 May 2017)           www.cog-genomics.org/plink/1.9/
(C) 2005-2017 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to UKB.log.
Options in effect:
  --allow-no-sex
  --bfile UKB_608_from23andme_PhenoUpdate
  --ld chr1:155238570:C:G chr1:155236376:C:T
  --out UKB
  --r2

515436 MB RAM detected; reserving 257718 MB for main workspace.
608 variants loaded from .bim file.
45857 people (0 males, 0 females, 45857 ambiguous) loaded from .fam.
Ambiguous sex IDs written to UKB.nosex .
45857 phenotype values loaded from .fam.
Using up to 127 threads (change this with --threads).
Before main variant filters, 45857 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.999064.
608

In [ ]:
cat UKB.ld

In [ ]:
# extract variant info from datasets
## D179H
grep 'chr1:155238570:C:G' UKB_608_from23andme_PhenoUpdate_GENO.model
## E365K
grep 'chr1:155236376:C:T' UKB_608_from23andme_PhenoUpdate_GENO.model

# xx/xx/xx = homozygous alternative(non-ref)/heterozygous/homozygous reference

In [ ]:
# extract frequency
module load plink/1.9.0-beta4.4
plink --bfile UKB_608_from23andme_PhenoUpdate --freq --out UKB_freq

In [ ]:
## head
head -n 1 UKB_freq.frq
## D179H
grep 'chr1:155238570:C:G' UKB_freq.frq
## E365K
grep 'chr1:155236376:C:T' UKB_freq.frq

In [ ]:
# LD estimate in AMP-PD
module load plink/1.9.0-beta4.4
plink --bfile AMP_282_from23andme --r2 --ld 'chr1:155238570:C:G' 'chr1:155236376:C:T' --out AMP

In [ ]:
ls Meta*

In [ ]:
# extract variant info from datasets
## D179H
grep 'chr1:155238570:C:G' AMP_282_from23andme_pheno_GENO.model
## E365K
grep 'chr1:155236376:C:T' AMP_282_from23andme_pheno_GENO.model

# xx/xx/xx = homozygous alternative(non-ref)/heterozygous/homozygous reference

In [ ]:
# extract frequency
module load plink/1.9.0-beta4.4
plink --bfile AMP_282_from23andme --freq --out AMP_freq

In [ ]:
## head
head -n 1 AMP_freq.frq
## D179H
grep 'chr1:155238570:C:G' AMP_freq.frq
## E365K
grep 'chr1:155236376:C:T' AMP_freq.frq

### 13.6 Maximum credible allele frequency

### 13.7 Write new files

In [ ]:
head Power_file_OR1_OR2_OR3_Jan5th2023.txt

In [ ]:
# New results file
candidates = data %>% filter(`Power_at_alpha005_OR2` >=0.8 & `P-value`<7.36377025036819e-05 |  `Power_at_alpha005_OR2` >=0.8 & `P-value`>7.36377025036819e-05 & `P-value` <0.5 | `Power_at_alpha005_OR2` <0.8 & `P-value`<7.36377025036819e-05) %>% tally()

In [1]:
pwd

/data/CARD/projects/Rare_variants_2023_VP


In [2]:
scp pitzv2@helix.nih.gov:/data/CARD/projects/Rare_variants_2023_VP/Meta_analysis_annotations_statistics_penetrance_power.txt /Users/pitzv2/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/Projects/23andme_rare_variants/Revision_NPJ/Final_files
scp pitzv2@helix.nih.gov:/data/CARD/projects/Rare_variants_2023_VP/Meta_analysis_minor_allele_counts.txt /Users/pitzv2/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/Projects/23andme_rare_variants/Revision_NPJ/Final_files
scp pitzv2@helix.nih.gov:/data/CARD/projects/Rare_variants_2023_VP/Results_Bonferroni_PoweratOR2.txt /Users/pitzv2/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/Projects/23andme_rare_variants/Revision_NPJ/Final_files

ssh: Could not resolve hostname nih.gov: Name or service not known


: 1

### 13.8 Change results layout based on power

In [ ]:
data = fread("Meta_results_679_master_file_revision.txt")
data %>% group_by(src) %>% tally() %>% mutate(perc = n/sum(n)*100)

In [ ]:
# filter down to those with power calculations
data = data %>% filter(!is.na(Power_at_alpha005_OR2))

In [ ]:
data %>% group_by(CLNSIG) %>% tally() %>% mutate(perc = n/sum(n)*100) %>% arrange(-n)

In [ ]:
genes = data %>% group_by(Gene) %>% tally() %>% mutate(perc = n/sum(n)*100)%>% arrange(-n)
write.table(genes, "List_of_all_genes.txt", row.names = F, sep = "\t", quote = F)

In [ ]:
scp pitzv2@helix.nih.gov:/data/CARD/projects/Rare_variants_2023_VP/List_of_all_genes.txt /Users/pitzv2/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/Projects/23andme_rare_variants/Revision_NPJ/Final_files
scp pitzv2@helix.nih.gov:/data/CARD/projects/Rare_variants_2023_VP/Meta_results_679_master_file_revision.txt /Users/pitzv2/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/Projects/23andme_rare_variants/Revision_NPJ/Final_files

#### 13.8.1 sufficient power at OR=2

In [ ]:
data %>% filter(Power_at_alpha005_OR2 >= 0.8) %>% tally()
# 162/669=24.2%

data %>% filter(Power_at_alpha005_OR2 >= 0.8) %>% group_by(CLNSIG) %>% tally() %>% mutate(perc = n/sum(n)*100)
A tibble: 5 × 3
CLNSIG	n	perc
<chr>	<int>	<dbl>
Benign	47	29.012346
Conflicting/Uncertain/Unknown	105	64.814815
Pathogenic	6	3.703704
Pathogenic/Risk_factor	2	1.234568
Risk_factor	2	1.234568

In [2]:
pwd


/data/CARD/projects/Rare_variants_2023_VP


#### 13.8.2 not sufficient power at OR=2

In [7]:
pwd

/data/CARD/projects/Rare_variants_2023_VP


In [6]:
scp pitzv2@helix.nih.gov:/data/CARD/projects/Rare_variants_2023_VP/Forest_plot_nopowerOR2.png /Users/pitzv2/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/Projects/23andme_rare_variants/Revision_NPJ/Final_files
scp pitzv2@helix.nih.gov:/data/CARD/projects/Rare_variants_2023_VP/Results_Bonferroni_NoPoweratOR2.txt /Users/pitzv2/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/Projects/23andme_rare_variants/Revision_NPJ/Final_files

7 Results_Bonferroni_NoPoweratOR2.txt


In [ ]:
# Power at OR=2

data %>% filter(is.na(Power_at_alpha005_OR2))  %>% select(MarkerName, VariantName, allfreq, AF_nfe, TOTAL_MAC_cases, TOTAL_MAC_ctrls)

A data.table: 10 × 6
MarkerName	VariantName	allfreq	AF_nfe	TOTAL_MAC_cases	TOTAL_MAC_ctrls
<chr>	<chr>	<dbl>	<chr>	<int>	<int>
chr1:16986321	ATP13A2_R1148H	NA	0	0	1
chr15:61961728	VPS13C_I1257V	NA	0	1	4
chr6:161569357	PRKN_Q311X	NA	.	0	0
chr14:22875844	LRP10_N299S	NA	.	0	1
chr21:32639773	SYNJ1_P1238S	NA	.	0	2
chr3:132522832	DNAJC13_R1893Q	NA	.	1	2
chr21:32701973	SYNJ1_R106W	NA	.	0	1
chr3:184327628	EIF4G1_A1236V	NA	0	0	4
chr1:16986208	ATP13A2_G1085E	NA	.	0	0
chr15:61984004	VPS13C_A577V	NA	0	0	1


# calculate them manually


In [1]:
pwd

/data/CARD/projects/Rare_variants_2023_VP


In [ ]:
### GLUD2 variant

In [4]:
grep "121049176" UKB_608_from23andme_PhenoUpdate_GENO.txt
grep "121049176" AMP_282_from23andme_pheno_GENO.txt

head -n 1 results_combined_23andMe.txt
grep "121049176" results_combined_23andMe.txt

grep: UKB_608_from23andme_PhenoUpdate_GENO.txt: No such file or directory
grep: AMP_282_from23andme_pheno_GENO.txt: No such file or directory
assay.name	scaffold	position	alleles	pvalue	pval.unadj	effect	stderr	pass	src	dose.b	AA.0	AB.0	BB.0	im.num.0	dose.b.0	AA.1	AB.1	BB.1	dose.b.1	im.num.1	is.v1	is.v2	is.v3	is.v4	is.v5	gt.rate	hw.p.value	p.date	freq.b	avg.rsqr	min.rsqr	p.batch
rs9697983	chrX	121049176	G/T	0.445079031345993	0.440383554839232	-0.0182062167879176	0.0237319236009759	TRUE	I	0.975521698594093	966	1349	64455	1532737	1.95097365424228	54	67	3718	1.95085609558383	12517.5	FALSE	FALSE	TRUE	FALSE	FALSE	0.895637605469783	1.4153659380274e-07	0.85840750882487	0.977136481846681	0.979696094989777	0.938883066177368	5.13236845035346e-06


In [8]:
grep 'L1795F' Meta_results_679_AMP_UKB_23andMe_annotation_counts_revision.txt

chr12:40322386	t	g	0.9999	0	0.9998	0.9999	-0.9109	0.2097	1.397e-05	-?-	52.1	2.087	1	0.1486	40322386	G	T	exonic	LRRK2	.	nonsynonymous SNV	LRRK2:NM_198578:exon37:c.G5385T:p.L1795F	rs111910483	47814	Parkinson_disease_8,_autosomal_dominant|not_provided	MONDO:MONDO:0011764,MedGen:C1846862,OMIM:607060|MedGen:CN517202	criteria_provided,_single_submitter	Conflicting/Uncertain/Unknown	.	.	.	.	.	.	.	.	.	.	.	.	.	.	.	.	.	LRRK2_L1795F	chr12:40322386	rs111910483	G/T	0.00027541352522454	0.000237825823345977	0.868630990685036	0.211719085084283	TRUE	I	0.000311791896820068	620832	101	0	1532737	0.000628566708430991	4350	0	0	0.00106454884730005	12517.5	FALSE	FALSE	FALSE	TRUE	FALSE	0.999784651951752	1	0.211616095431841	8.47914652366555e-05	0.849218487739563	0.838042914867401	0.255805737850563	4350	0	0	620832	101	0	4350	620933	0	101	0	0	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	0	2	1449	0	0	2556	1451	2556	2	0	5799	2	0	623388	101	0	5801	623489	2	101	0.000344768143423548	0.000161991630967026	9.9999999999989e-05	4.2566167

In [1]:
grep "D179H" Meta_results_679_AMP_UKB_23andMe_annotation_counts_revision.txt

chr1:155238570	c	g	1e-04	0	1e-04	1e-04	1.1863	0.2163	4.143e-08	?++	0	0.182	1	0.6695	155238570	C	G	exonic	GBA	.	nonsynonymous SNV	GBA:NM_001171811:exon4:c.G274C:p.D92H,GBA:NM_001171812:exon4:c.G388C:p.D130H,GBA:NM_000157:exon5:c.G535C:p.D179H,GBA:NM_001005741:exon6:c.G535C:p.D179H,GBA:NM_001005742:exon6:c.G535C:p.D179H	rs147138516	.	.	.	.	Conflicting/Uncertain/Unknown	6.391e-05	0.0001	5.739e-05	7.212e-05	6.366e-05	0	.	0	0	0.0001	0	0	0	0.0002	7.364e-05	.	0.0002	GBA_D179H	chr1:155238570	rs147138516	C/G	4.26925619013227e-06	3.3942529979508e-06	1.21310434188877	0.225231377391631	TRUE	I	0.000195622444152832	3054396	740	0	1532737	0.000382138388424644	22820	12	0	0.000941897030431905	12517.5	FALSE	FALSE	TRUE	TRUE	TRUE	0.998777171999375	1	7.5413637304533e-46	0.000124064524961165	0.865636765956879	0.849082350730896	0.256638323471983	22820	12	0	3054396	740	0	22832	3055136	12	740	0	0	0	4	7802	0	8	38042	7806	38050	4	8	0	0	1451	0	0	2556	1451	2556	0	0	32073	16	0	3094994	748	0	32089	3095742	16	748	0.00

In [7]:
grep '155238570' UKB_608_from23andme_PhenoUpdate_GENO.model

1	chr1:155238570:C:G	G	C	GENO	0/4/7802	0/8/38042	NA	NA	NA


In [6]:
grep '155238570'  AMP_282_from23andme_pheno_GENO.model

1	chr1:155238570:C:G	G	C	GENO	0/0/1451	0/0/2556	NA	NA	NA


In [9]:
grep '155238570' results_combined_23andMe.txt

rs147138516	chr1	155238570	C/G	4.26925619013227e-06	3.3942529979508e-06	1.21310434188877	0.225231377391631	TRUE	I	0.000195622444152832	3054396	740	0	1532737	0.000382138388424644	22820	12	0	0.000941897030431905	12517.5	FALSE	FALSE	TRUE	TRUE	TRUE	0.998777171999375	1	7.5413637304533e-46	0.000124064524961165	0.865636765956879	0.849082350730896	0.256638323471983


In [10]:
pwd

/data/CARD/projects/Rare_variants_2023_VP


#### 13.8.3 Total new plots

In [ ]:
results1 = results %>% filter(Results == "Results_1")
plot1 = ggplot(results1, mapping = aes(x= OR, y = reorder(VariantName, -OR)))+
  geom_vline(aes(xintercept =1), size = .5, linetype = "dashed")+
  geom_errorbarh(aes(xmax = U95, xmin = L95), size = .5, height = .2) +
  geom_point(size = 3.5, aes(color = CLNSIG)) +
  scale_x_continuous(breaks = seq(0,14,2), labels = seq(0,14,2), limits = c(0,14)) +
  theme_bw()+
  theme(panel.grid.minor = element_blank()) +
  ylab("Variants")+
  xlab("OR and 95% CI")+
  ggtitle("Sufficiently powered and passing multiple test correction")+
  theme(plot.title = element_text(hjust=0.5)) +
 theme(plot.title = element_text(hjust=0.5),
       legend.position="bottom")
       
ggsave("Forest_plot_Top5_variants.png", plot1, width = 8, height = 5, dpi=300, units = "in")

In [ ]:
results2 = results %>% filter(Results == "Tier_2")

plot2 = ggplot(results2, mapping = aes(x= log(OR), y = reorder(VariantName, -OR)))+
  geom_vline(aes(xintercept =0), size = .5, linetype = "dashed")+
  geom_errorbarh(aes(xmax = log(U95), xmin = log(L95)), size = .5, height = .2) +
  geom_point(size = 3.5, aes(color = CLNSIG)) +
  scale_x_continuous(breaks = seq(0,5,1), labels = seq(0,5,1), limits = c(0,5)) +
  theme_bw()+
  theme(panel.grid.minor = element_blank()) +
  ylab("Variants")+
  xlab("OR and 95% CI on log scale")+
  ggtitle("Strong candidate variants based on power and p-value")+
  theme(plot.title = element_text(hjust=0.5)) +
 theme(plot.title = element_text(hjust=0.5),
       legend.position="bottom")
plot2
       
ggsave("Forest_plot_candidate_variants.png", plot2, width = 8, height = 5, dpi=300, units = "in")

In [4]:
head -n 1 Meta_results_679_master_file_revision.txt
grep E365K Meta_results_679_master_file_revision.txt

MarkerName	Allele1	Allele2	Freq1	FreqSE	MinFreq	MaxFreq	Effect	StdErr	P-value	Direction	HetISq	HetChiSq	HetDf	HetPVal	End	Ref	Alt	Func.refGene	Gene	GeneDetail.refGene	ExonicFunc.refGene	AAChange	avsnp150	CLNALLELEID	CLNDN	CLNDISDB	CLNREVSTAT	CLNSIG	AF	AF_popmax	AF_male	AF_female	AF_raw	AF_afr	AF_sas	AF_amr	AF_eas	AF_nfe	AF_fin	AF_asj	AF_oth	non_topmed_AF_popmax	non_neuro_AF_popmax	non_cancer_AF_popmax	controls_AF_popmax	VariantName	CHR.BP	assay.name	alleles	pvalue	pval.unadj	effect	stderr	pass	src	dose.b	AA.0	AB.0	BB.0	im.num.0	dose.b.0	AA.1	AB.1	BB.1	dose.b.1	im.num.1	is.v1	is.v2	is.v3	is.v4	is.v5	gt.rate	hw.p.value	p.date	freq.b	avg.rsqr	min.rsqr	p.batch	23andme_homref_cases	23andme_het_cases	23andme_homnonref_cases	23andme_homref_ctrls	23andme_het_ctrls	23andme_homnonref_ctrls	23andMe_n_cases	23andMe_n_ctrls	23andMe_MAC_cases	23andMe_MAC_ctrls	dose_MAC_case	dose_MAC_ctrls	UKB_homnonref_cases	UKB_het_cases	UKB_homref_cases	UKB_homnonref_ctrls	UKB_het_ctrls	UKB_homref_ctrls	UKB_n_case